## Downloads necessários

In [1]:
!pip install langchain langchain-google-genai pypdf
!pip install langchain-community

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


## Imports necessários

In [108]:
import textwrap
from langchain_community.chat_models import ChatOllama
from langchain.prompts import ChatPromptTemplate
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

## Definindo a IA

In [82]:
llm = ChatOllama(model="mistral", temperature=0)


## Pergunta que será feita para a IA

In [83]:
pergunta = "Qual era o nome do cavalo de napoleão ?"

## Como ela deve responder

In [86]:
contexto = "Responda a seguinte pergunta: "

prompt_tradicional = ChatPromptTemplate.from_template("{contexto}{pergunta}")

prompt_pronto = prompt_tradicional.format(contexto=contexto, pergunta=pergunta)
print(prompt_pronto)

Human: Responda a seguinte pergunta: Qual era o nome do cavalo de napoleão ?


## Resposta

In [88]:
chain_tradicional = prompt_tradicional | llm

resposta_tradicional = chain_tradicional.invoke({
    "contexto": contexto,
    "pergunta": pergunta
})

texto = str(resposta_tradicional.content)
print("Pergunta:", pergunta)
print("Resposta:", textwrap.fill(texto, width=75))

Pergunta: Qual era o nome do cavalo de napoleão ?
Resposta:  O cavalo de Napoleão Bonaparte era chamado de Marengo. Este famoso cavalo
foi um presente de seu irmão José Bonaparte, que era rei da Holanda na
época. O cavalo ficou conhecido por ter acompanhado Napoleão em muitas de
suas campanhas militares.


## Adicionando documentos

In [89]:
loader = PyPDFLoader(r"documentos\Perguntas Frequentes - Eu Vereador.pdf")
docs = loader.load()

# Junta o conteúdo de todas as páginas
doc_perguntas_frequentes = "\n".join(doc.page_content for doc in docs)

print(doc_perguntas_frequentes)

Perguntas Frequentes – Canal Eu Vereador 
 
1. Quem pode participar? 
2. O curso é só para candidatos? 
3. Qual é a carga horária e como é a programação? 
4. Onde e quando acontece o curso? 
5. O que está incluso na inscrição? 
6. O curso é prático ou teórico? 
7. O curso oferece certificado? 
8. Qual é o valor da inscrição? 
9. É possível pagar por nota de empenho? 
10. Quantas vagas estão disponíveis? 
11. Vai ter gravação das aulas? 
12. O curso ajuda diretamente na reeleição?


## Separa o texto em partes menores (chunk)

In [109]:
# chunk_size = tamanho máximo do pedaço
# chunk_overlap = quanto vai sobrepor de um pedaço para o outro (evita cortar no meio de ideias importantes)
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=800,  # Ajuste conforme o modelo que vai usar
    chunk_overlap=100
)
chunks = text_splitter.split_documents(docs)


## Pergunta que será feita para a IA

In [110]:
pergunta = "Qual a pergunta menos frequente ?"

## Como ela deve responder

In [111]:
contexto = doc_perguntas_frequentes

prompt_rag = ChatPromptTemplate.from_template("""
Use o contexto abaixo para responder a pergunta.
Se não souber a resposta baseado no contexto, diga que não tem a informação.

Contexto: {contexto}

Pergunta: {pergunta}""")

prompt_rag_pronto = prompt_rag.format(
    contexto=contexto,
    pergunta=pergunta
)
print(prompt_rag_pronto)

Human: 
Use o contexto abaixo para responder a pergunta.
Se não souber a resposta baseado no contexto, diga que não tem a informação.

Contexto: Perguntas Frequentes – Canal Eu Vereador 
 
1. Quem pode participar? 
2. O curso é só para candidatos? 
3. Qual é a carga horária e como é a programação? 
4. Onde e quando acontece o curso? 
5. O que está incluso na inscrição? 
6. O curso é prático ou teórico? 
7. O curso oferece certificado? 
8. Qual é o valor da inscrição? 
9. É possível pagar por nota de empenho? 
10. Quantas vagas estão disponíveis? 
11. Vai ter gravação das aulas? 
12. O curso ajuda diretamente na reeleição?

Pergunta: Qual a pergunta menos frequente ?


## Resposta

In [113]:
chain_rag = prompt_rag | llm

resposta_rag = chain_rag.invoke({
    "contexto": contexto,
    "pergunta": pergunta
})

texto = str(resposta_rag.content)
print("Pergunta:", pergunta)
print("Resposta:", textwrap.fill(texto, width=75))

Pergunta: Qual a pergunta menos frequente ?
Resposta:  A pergunta menos frequente entre as listadas no contexto é a seguinte: "O
curso ajuda diretamente na reeleição?"


## Memória

In [ ]:
# ConversationalRetrievalChain

## Guardrails